In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.parse
import urllib.request
import time

forms = ["Plain", "Past plain", "Present polite", "Past polite", "Potential",
"Te", "Imperative", "Volitional", "Ba", "Causative", "Chau",
"Chimau", "I", "Nasai imperative", "Passive", "Tara"]

df_headers = ['Kanji', 'Plain', 'Plain negative', 'Past plain', 'Past plain negative', 'Present polite',
           'Present polite negative',
           'Past polite', 'Past polite negative', 'Potential', 'Potential negative', 'Te',
           'Te negative', 'Imperative', 'Imperative negative', 'Volitional', 'Volitional negative', 
           'Ba', 'Ba negative', 'Causative', 'Causative negative', 'Chau', 'Chau negative', 'Chimau', 'Chimau negative', 'I', 'I negative', 'Nasai imperative',
           'Nasai imperative negative', 'Passive', 'Passive negative', 'Tara', 'Tara negative']


headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36',
           'cookie': 'visid_incap_1712983=Dfo+CVLdTCSzMHcIXEgPSb1y7l0AAAAAQUIPAAAAAAAetg/EHDkFJUigbIo4eaK4; incap_ses_532_1712983=dFrgDpkdqkYCF1u+mQxiB71y7l0AAAAAksMc42V5CJx6OdUZdeHflA==; has_js=1; _ga=GA1.2.626207180.1575908117; _gid=GA1.2.512657448.1575908117; __utma=158387685.626207180.1575908117.1575908117.1575908117.1; __utmc=158387685; __utmz=158387685.1575908117.1.1.utmcsr=(direct)^|utmccn=(direct)^|utmcmd=(none); __utmt=1; __utmt_b=1; _aeaid=5e12b9d6-0171-4fde-8ccf-1bba809a1bb2; aeatstartmessage=true; __utmb=158387685.4.10.1575908117',
          }

In [2]:
xls_path = "/content/en_vocab_list.xlsx"
sheet_names = ["Verbs", "Nouns", "Adjectives (-i)", "Adjectival nouns", "Adverbs", "Pronouns", "Determiners", "Auxiliaries"]
df = pd.read_excel(xls_path, sheet_name = sheet_names)

In [3]:
df_verbs = df["Verbs"]
df_nouns = df["Nouns"]
df_adj_i = df["Adjectives (-i)"]
df_adj_nouns = df["Adjectival nouns"]
df_adverbs = df["Adverbs"]
df_pronouns = df["Pronouns"]
df_determiners = df["Determiners"]
df_auxiliaries = df["Auxiliaries"]

df = None

In [19]:
def p(df, rank):
  temp = df[df["Frequency ranking"] < rank]
  return temp

temp = []
rank = 3000

temp_df = p(df_verbs, rank)
#temp_df = temp_df.append(p(df_nouns, rank))
#temp_df = temp_df.append(p(df_adj_i, rank))
#temp_df = temp_df.append(p(df_adj_nouns, rank))
#temp_df = temp_df.append(p(df_adverbs, rank))
#temp_df = temp_df.append(p(df_pronouns, rank))
#temp_df = temp_df.append(p(df_determiners, rank))

# temp_df = p(df_verbs, rank)
temp_df = temp_df.sort_values(by=["Frequency ranking"])
# temp_df.to_csv("verbs.csv", index=False)

In [259]:
temp_df

,Frequency ranking,Kanji,Standard Orthography (in newspapers),Standard Reading (in Hiragana),Part of speech
0,10,為る,する,する,動詞-非自立可能
1,13,居る,いる,いる,動詞-非自立可能
2,14,有る,ある,ある,動詞-非自立可能
3,17,言う,いう,いう,動詞-一般
4,23,成る,なる,なる,動詞-非自立可能
...,...,...,...,...,...
478,2968,差し出す,差し出す,さしだす,動詞-一般
479,2968,受け止める,受け止める,うけとめる,動詞-一般
480,2982,下す,下す,くだす,動詞-一般
481,2987,試みる,試みる,こころみる,動詞-一般


In [260]:
def get_conjucations(kanji):
  s = ""
  for i, c in enumerate(kanji.encode('utf-8').hex()):
    if i % 2 == 0:
      s += "%"
    s += c

  url = f"https://www.sljfaq.org/cgi/verb-inflect.cgi?verb={s}&ot=kana"
  url_request = urllib.request.Request(url, data=None, headers=headers)
  url_response = urllib.request.urlopen(url_request)
  soup = BeautifulSoup(url_response, "html.parser", from_encoding="iso-8859-1")

  temp_soup = soup
  table = temp_soup.find("table", {"class": "vocab-list bordered"})
  #symbol.text.encode('latin1').decode('utf-8')
  table_rows = table.find_all("td")

  temp_table = []
  for tr in table_rows:
    text = tr.text.encode('latin1').decode('utf-8').strip()
    text = text.replace("\n", "")
    if text in forms:
      temp_table.append([])

    temp_table[len(temp_table) - 1].append(text)

  temp_table_dict = {}
  for row in temp_table:
    temp_table_dict[row[0]] = row[1]
    temp_table_dict[row[0] + " negative"] = row[2]

  temp_table_dict["Kanji"] = kanji

  return temp_table_dict

In [261]:
ok_df = pd.DataFrame(columns=df_headers)
for kanji in temp_df["Standard Orthography (in newspapers)"]:
  try:
    time.sleep(1)
    ok_df = ok_df.append(get_conjucations(kanji), ignore_index=True)
  except:
    print(kanji)

下さる
置く／おく
対する
感ずる
関する
信ずる
生ずる
通ずる
応ずる
張る／貼る
愛する
要する
達する
命ずる
有する
発する
属する
接する
称する
存ずる
論ずる
反する


In [265]:
ok_df.to_csv("verbs.csv", index=False)

In [262]:
df_verbs.columns = ['Frequency ranking', 'Kanji', 'Standard Orthography (in newspapers)',
       'Standard Reading (in Hiragana)', 'Part of speech']
#temp = pd.merge(df1, df2, on="movie_title")

In [263]:
temp = pd.merge(ok_df, df_verbs, on="Kanji")

In [264]:
temp.to_csv("verbs_final.csv", index=False)

In [171]:


kanji = "為る" # .encode('utf-8').strip()
s = ""
for i, c in enumerate(kanji.encode('utf-8').hex()):
  if i % 2 == 0:
    s += "%"
  s += c

url = f"https://www.sljfaq.org/cgi/verb-inflect.cgi?verb={s}&ot=kana"
url_request = urllib.request.Request(url, data=None, headers=headers)
url_response = urllib.request.urlopen(url_request)
soup = BeautifulSoup(url_response, "html.parser", from_encoding="iso-8859-1")

In [176]:
temp_soup = soup
table = temp_soup.find("table", {"class": "vocab-list bordered"})
#symbol.text.encode('latin1').decode('utf-8')
table_rows = table.find_all("td")

In [210]:



temp_table = []
for tr in table_rows:
  text = tr.text.encode('latin1').decode('utf-8').strip()
  text = text.replace("\n", "")
  if text in forms:
    temp_table.append([])

  temp_table[len(temp_table) - 1].append(text)

temp_table_dict = {}
for row in temp_table:
  temp_table_dict[row[0]] = row[1]
  temp_table_dict[row[0] + " negative"] = row[2]


  # print(tr.text.encode('latin1').decode('utf-8'))
ok_df.append(temp_table_dict, ignore_index=True)

,Plain,Plain negative,Past plain,Past plain negative,Present polite,Present polite negative,Past polite,Past polite negative,Potential,Potential negative,Te,Te negative,Imperative,Imperative negative,Volitional,Volitional negative,Ba,Ba negative,Causative,Causative negative,Chau,Chau negative,Chimau,Chimau negative,I,I negative,Nasai imperative,Nasai imperative negative,Passive,Passive negative,Tara,Tara negative
0,なる,ならない,なった,ならなかった,なります,なりません,なりました,なりません でした,なれる,なれない,なって,ならないで,なれ,なる な,なろう,,なれば,ならなければ,ならせる,ならせない,なっちゃう,,なっちまう,,なり,,なりなさい,,なられる,なられない,なったら,ならなかったら


In [215]:
temp_df["Lexeme"][:3]

0    為る
1    居る
2    有る
Name: Lexeme, dtype: object

In [211]:
temp_table_2

[['Plain', 'なる'],
 ['Plain negative', 'ならない'],
 ['Past plain', 'なった'],
 ['Past plain negative', 'ならなかった'],
 ['Present polite', 'なります'],
 ['Present polite negative', 'なりません'],
 ['Past polite', 'なりました'],
 ['Past polite negative', 'なりません でした'],
 ['Potential', 'なれる'],
 ['Potential negative', 'なれない'],
 ['Te', 'なって'],
 ['Te negative', 'ならないで'],
 ['Imperative', 'なれ'],
 ['Imperative negative', 'なる な'],
 ['Volitional', 'なろう'],
 ['Volitional negative', ''],
 ['Ba', 'なれば'],
 ['Ba negative', 'ならなければ'],
 ['Causative', 'ならせる'],
 ['Causative negative', 'ならせない'],
 ['Chau', 'なっちゃう'],
 ['Chau negative', ''],
 ['Chimau', 'なっちまう'],
 ['Chimau negative', ''],
 ['I', 'なり'],
 ['I negative', ''],
 ['Nasai imperative', 'なりなさい'],
 ['Nasai imperative negative', ''],
 ['Passive', 'なられる'],
 ['Passive negative', 'なられない'],
 ['Tara', 'なったら'],
 ['Tara negative', 'ならなかったら']]

In [20]:
temp_df.columns = ['Frequency ranking', 'Lexeme', 'Kanji',
       'Standard Reading (in Hiragana)', 'Part of speech']

In [21]:
df = pd.read_csv("/content/verbs (1).csv")

In [88]:
# temp = pd.merge(df, temp_df, on="Kanji")
temp = []

temp3 = []
temp4 = []

last = 0
for curr in range(0, len(df)):
  for i in range(last, len(temp_df)):
    if df.loc[curr]["Kanji"] == temp_df.loc[i]["Kanji"]:
      temp3.append(df.loc[curr])
      temp4.append(temp_df.loc[i])
      #temp.append([df.loc[curr], temp_df.loc[i]])
      break

    last = i

temp3 = pd.DataFrame(temp3)
temp4 = pd.DataFrame(temp4)

In [89]:
len(df)

461

In [105]:
len(temp3)
temp4.columns

Index(['Frequency ranking', 'Lexeme', 'Kanji',
       'Standard Reading (in Hiragana)', 'Part of speech'],
      dtype='object')

In [106]:
temp3["Frequency ranking"] = temp4["Frequency ranking"]
temp3["Lexeme"] = temp4["Lexeme"]
temp3["Standard Reading (in Hiragana)"] = temp4["Standard Reading (in Hiragana)"]

In [107]:
len(temp3)

461

In [108]:
temp3

,Kanji,Plain,Plain negative,Past plain,Past plain negative,Present polite,Present polite negative,Past polite,Past polite negative,Potential,Potential negative,Te,Te negative,Imperative,Imperative negative,Volitional,Volitional negative,Ba,Ba negative,Causative,Causative negative,Chau,Chau negative,Chimau,Chimau negative,I,I negative,Nasai imperative,Nasai imperative negative,Passive,Passive negative,Tara,Tara negative,Frequency ranking,Lexeme,Standard Reading (in Hiragana)
0,する,する,しない,した,しなかった,します,しません,しました,しません でした,できる,NaN,して,しないで,しろ,する な,しよう,NaN,すれば,しなければ,させる,させない,しちゃう,NaN,しちまう,NaN,し,NaN,しなさい,NaN,される,NaN,したら,しなかったら,10.0,為る,する
1,いる,いる,いらない,いった,いらなかった,いります,いりません,いりました,いりません でした,いれる,いれない,いって,いらないで,いれ,いる な,いろう,NaN,いれば,いらなければ,いらせる,いらせない,いっちゃう,NaN,いっちまう,NaN,いり,NaN,いりなさい,NaN,いられる,いられない,いったら,いらなかったら,13.0,居る,いる
2,ある,ある,ない,あった,なかった,あります,ありません,ありました,ありません でした,あれる,あれない,あって,ないで,あれ,ある な,あろう,NaN,あれば,なければ,あらせる,あらせない,あっちゃう,NaN,あっちまう,NaN,あり,NaN,ありなさい,NaN,あられる,あられない,あったら,なかったら,14.0,有る,ある
3,いう,いう,いわない,いった,いわなかった,いいます,いいません,いいました,いいません でした,いえる,いえない,いって,いわないで,いえ,いう な,いおう,NaN,いえば,いわなければ,いわせる,いわせない,いっちゃう,NaN,いっちまう,NaN,いい,NaN,いいなさい,NaN,いわれる,いわれない,いったら,いわなかったら,17.0,言う,いう
4,なる,なる,ならない,なった,ならなかった,なります,なりません,なりました,なりません でした,なれる,なれない,なって,ならないで,なれ,なる な,なろう,NaN,なれば,ならなければ,ならせる,ならせない,なっちゃう,NaN,なっちまう,NaN,なり,NaN,なりなさい,NaN,なられる,なられない,なったら,ならなかったら,23.0,成る,なる
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,差し出す,さしだす,さしださない,さしだした,さしださなかった,さしだします,さしだしません,さしだしました,さしだしません でした,さしだせる,さしだせない,さしだして,さしださないで,さしだせ,さしだす な,さしだそう,NaN,さしだせば,さしださなければ,さしださせる,さしださせない,さしだしちゃう,NaN,さしだしちまう,NaN,さしだし,NaN,さしだしなさい,NaN,さしだされる,さしだされない,さしだしたら,さしださなかったら,2863.0,放つ,はなつ
457,受け止める,うけとめる,うけとめない,うけとめた,うけとめなかった,うけとめます,うけとめません,うけとめました,うけとめません でした,うけとめられる,うけとめられない,うけとめて,うけとめないで,うけとめよ,うけとめる な,うけとめよう,NaN,うけとめれば,うけとめなければ,うけとめさせる,うけとめさせない,うけとめちゃう,NaN,うけとめちまう,NaN,うけとめ,NaN,うけとめなさい,NaN,うけとめられる,うけとめられない,うけとめたら,うけとめなかったら,2873.0,勤める,つとめる
458,下す,おろす,おろさない,おろした,おろさなかった,おろします,おろしません,おろしました,おろしません でした,おろせる,おろせない,おろして,おろさないで,おろせ,おろす な,おろそう,NaN,おろせば,おろさなければ,おろさせる,おろさせない,おろしちゃう,NaN,おろしちまう,NaN,おろし,NaN,おろしなさい,NaN,おろされる,おろされない,おろしたら,おろさなかったら,2875.0,解く,とく
459,試みる,こころみる,こころみない,こころみた,こころみなかった,こころみます,こころみません,こころみました,こころみません でした,こころみられる,こころみられない,こころみて,こころみないで,こころみよ,こころみる な,こころみよう,NaN,こころみれば,こころみなければ,こころみさせる,こころみさせない,こころみちゃう,NaN,こころみちまう,NaN,こころみ,NaN,こころみなさい,NaN,こころみられる,こころみられない,こころみたら,こころみなかったら,2877.0,浴びる,あびる


In [57]:
df.loc[curr]["Kanji"]

'載せる'

In [58]:
temp_df.loc[i]["Kanji"]

'載せる'

In [109]:
temp3.to_csv("actual_verbs_final.csv", index=False)